# 📈 ETH/USD Price Prediction & Trade Signal Generator
Este notebook permite entrenar, actualizar y usar un modelo RNN para predecir el precio de ETH/USD.
Además, genera señales de trading basadas en predicciones futuras (comprar, vender o mantener).

In [1]:
# 🔧 Librerías
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
from datetime import datetime


2025-04-13 21:49:02.209370: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-13 21:49:02.219142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744573742.230828   10210 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744573742.234540   10210 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744573742.243159   10210 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# ⚙️ Función para cargar los datos históricos
def load_data(filepath):
    coin="ETH"
    df_coin = pd.read_csv(filepath,
                          parse_dates=['date'],
                          skiprows=1,
                          skipfooter=1)
    df_coin = df_coin[['date', 'open']]
    df_coin = df_coin.set_index('date')
    df_coin = df_coin.sort_index()
    df_coin = df_coin.rename(columns={'open': coin})
    return df_coin


In [3]:
def create_windows_np(data, window_size, horizon, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy.

    Parameters:
    data (np.ndarray): Time series data with one dimension.
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict.
    shuffle (bool): Shuffle the windows or not.

    Returns:
    tuple: A tuple containing the input-output pairs (windows, targets) as NumPy arrays.
    """

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size+horizon-1])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y


In [4]:
def normalize_data(train_data):
    mu = train_data.mean()
    sigma = train_data.std()

    train_data = (train_data - mu) / sigma

    return train_data, mu, sigma

In [34]:
# 🧠 Crear el modelo RNN
def build_model(window_size):
    inputs = keras.Input(shape=(window_size, 1))
    x = layers.Conv1D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Bidirectional(layers.LSTM(16, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(16, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(16))(x)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
    return model

In [5]:
# 🔄 Función para actualizar el modelo con nuevos datos
def update_model(model, new_data, window_size):
    X_new, y_new = create_windows_np(new_data, window_size)
    X_new = X_new.reshape((X_new.shape[0], X_new.shape[1], 1))
    model.fit(X_new, y_new, epochs=5, batch_size=32, verbose=0)
    return model

In [44]:
# 🔮 Hacer predicción futura
def predict_next_price(model, last_window):
    return model.predict(last_window)[0][0]

In [48]:
# 📊 Generar señal de trading según predicción
def generate_trade_signal(current_price, predicted_price, threshold=0.03):
    change = (predicted_price - current_price) / current_price
    print(change)
    if change > threshold:
        return 'BUY'
    elif change < -threshold:
        return 'SELL'
    else:
        return 'HOLD'

In [54]:
# 🧪 Ejemplo de uso
window_size = 12
horizon = 48
filepath = '/home/eprdz/cryptoAI/Gemini_ETHUSD_1h.csv'  # Ruta al archivo con datos históricos

# Cargar datos
df = load_data(filepath)
series = df['ETH'].values
X_norm, mu, sigma = normalize_data(series)

# Crear datos de entrenamiento
X, y = create_windows_np(X_norm, window_size, horizon)
#X = X.reshape((X.shape[0], X.shape[1], 1))

# Crear o cargar modelo
#model = build_model(window_size)
model = keras.models.load_model('/home/eprdz/cryptoAI/eth.keras')  # Descomentar si ya tienes uno entrenado

# Entrenar
#model.fit(X, y, epochs=10, batch_size=64, validation_split=0.2)
#model.save('eth_rnn_model.h5')

/tmp/ipykernel_10210/4157321808.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df_coin = pd.read_csv(filepath,


In [34]:
X.shape

(77804, 12)

In [25]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 12, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 10, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 5, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 5, 32)          │         6,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 5, 32)          │         6,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 32)             │         6,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,933 (222.40 KB)

 Trainable params: 18,977 (74.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 37,956 (148.27 KB)

In [61]:
X.shape

(77804, 12)

In [64]:
# 🚀 Predecir y generar señal
last_window = X[-window_size:]
current_price = X_norm[-1] * sigma + mu
predicted_price = predict_next_price(model, last_window) * sigma + mu
signal = generate_trade_signal(current_price, predicted_price)

print(f'Current price: {current_price:.2f}')
print(f'Predicted next price: {predicted_price:.2f}')
print(f'Trade signal: {signal}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.02682572935836995
Current price: 2007.16
Predicted next price: 2061.00
Trade signal: HOLD
